# Queensland Spatial Information Canonical Examples Scenario Demonstrator
## For the [FrontierSI](https://frontiersi.com.au) Queensland Land Information Project
### By Nicholas J. Car

## 1. Introduction

This Workbook is a _Scenario Demonstrator_ for the _QSI Cadastre Address Project_. It lists some of that project's Requirements and shows how the models proposed for it cater for them.

The goal of this Scenario Demonstrator is not to implement Canonical Examples and tests for _all_ Requirements but just enough to indicate the likely viability of the models as per Requirements.

It shows model skipp per Requirements by presenting _Canonical Examples_ which are data files stored in this Notebook's repository, made according to the project models and then showing validation of and queries against those examples.

The project models are documented within the [Queensland Land Information Supermodel](https://nicholascar.com/qli-supermodel/supermodel.html) and the Canonical Example data files are named according to the Requirements they are for and stored in the [canonical-examples/ folder](https://github.com/nicholascar/qsi-sd-canonical/tree/master/canonical-examples).

### 1.1 Executable Workbook

This Workbook is a piece of documented Python code that can be run online, in a browser, step-by-step, without the user having to install anything. The code sections you see below can be run, results seen, altered and then re-run, to allow users to really get to grips with the data, queries and so on. Please feel free to experiment: changes are not stored, so you can't really break anything!

### 1.2 Canonical Examples data format
The Canonical Examples data is [RDF](https://www.w3.org/RDF/) data, that is data made according to a conceptual model used to create [_Knowledge Graphs_](https://www.ontotext.com/knowledgehub/fundamentals/what-is-a-knowledge-graph/) which are data models of nodes representing ideas connected by edges of defined relationships.

RDF data is system-independent - it can be stored and used in many different systems - but it is validatable and queryable too, using a standard query language called [SPARQL](https://www.w3.org/TR/sparql11-query/) which is similar to SQL, but for graphs.

### 1.3 Validators & Validation

Validators provided with all the models in this project are used here to show what canonical examples do and do not conform to the models. the validators are written using the [SHACL](https://www.w3.org/TR/shacl/) constraints language and are executable in this workbook by use of the [pySHACL](https://pypi.org/project/pyshacl/) tool.

Validators are also data files with individual constraint checks implemented as _Shapes_ which are that is a graph pattern test functions. Pretty much anything in graph data van be validated using a Shape. Shapes and Requirements are not necessarily 1:1, so Requirements might list multiple Shapes needed to assure their handing in data.

The validator, Canonical Example data and background models are all added to a call to the tool and results - true/false valid and if invalid, details - are able to be printed.

### 1.4 Queries

SPARQL queries can be run against Canonical Examples by loading the example files and using the [rdflib](https://pypi.org/project/rdflib/) tool to pose queries and return results for printing.

### 1.5 Initial Requirement Explained

The first Requirement demonstrated here, A18 in Section 2.1, is explained in more detail than subsequent Requirements to ensure all details of the technical implementation are understood.

## 2. Requirements

### 2.1 Req. A18

#### 2.1.1 Context
Requirement A18 is an Address Requirement with the following details (from the _QSI Cadastre Address Requirements_ list):

**ID** | **Statement** | **Relevant Model Elements** | **Canonical Examples** | **Scenario Demonstrator**
--- | --- | --- | --- | ---
A18 | Ideally entities as address components that are maintained elsewhere e.g. locality (and Local Government Area even though this isn't part of an official address string) should be linked to rather than repeated in the addressing database. | Address Model Shapes sh-01, sh-02, Overarching Model, a18 | A18-v01, A18-i01, A18-i02 | Notebook SD01, EG01

This Requirement requires two models to be satisfied:

1. [Address Model](https://nicholascar.com/anz-nat-addr-model-candidate/model.html) - to ensure an Address object has the properties required
2. QLI Supermodel's [Overarching Model](https://nicholascar.com/qli-supermodel/supermodel.html#_overarching_model) - since the "...entities...maintained elsewhere e.g. locality..." are objects for which there is no Component Model but for which the Overarching Model has a generic slot

The Address Model contains an `Address` class and a property for it to indicate `AddressComponent` class instances which is `hasAddressComponent`. `AddressComponent` objects then contain a `hasComponentType`, a `hasValue`  and `hasValueText` properties, see the [`AddressComponent` class documentation](https://nicholascar.com/anz-nat-addr-model-candidate/model.html#AddressComponent).

The Overarching Model contains an `AdministrativeArea` class which will be what the Supermodel considers Localities and other such entities to be. Instances of this class will have identifiers and their class will be defined.

#### 2.1.2 Demonstration Method

Can data be formulated according to the various models, some of which is and some of which is not valid according Requirement A18? Are such datum able to be tested to prove validity or invalidity?

The Address Model's [validator](http://w3id.org/profile/anz-address/validator) ([also stored locally](https://github.com/nicholascar/qsi-sd-canonical/blob/master/validators/addr.shacl.ttl)) has 2 relevant Shapes, `sh-01`, `sh-02` and the Overarching Model's [validator]() ([also stored locally](https://github.com/nicholascar/qsi-sd-canonical/blob/master/validators/qsi-over.shacl.ttl)) has one `a18`, all of which are , listed in the Requirements details table above.

Shape `sh-02`, as an example of a Shape, is as follows:

```turtle
<sh-02>
	a sh:NodeShape ;
	sh:targetObjectsOf addr:hasAddressComponent ; # addr:AddressComponent instances
	sh:sparql [
		a sh:SPARQLConstraint ;
		sh:message "The hasValue property of an AddressComponent with addressComponentType isov1:locality must indicate an IRI (of a Concept from a vocabulary), not text" ;
		sh:prefixes addr: ;
		sh:select """
			SELECT $this ?value
			WHERE {
                $this addr:hasComponentType isov1:locality .
                $this addr:hasValue ?value .
                FILTER (!isIRI(?value))
			}
			""" ;
	] ;
.
```

This Shape checks that each instance of `AddressComponent` that is of type `locality` (according to [an extended version of ISO's Address Component Types Vocabulary](https://nicholascar.com/anz-nat-addr-model-candidate/model.html#_address_component_type_vocabulary)) has a value that is an IRI - a web address - and not a numerical or textual value.

Separately to this validator, a mapping between validator Shapes and Requirements is maintained within [the QLI Supermodel's listing of Requirements](https://nicholascar.com/qli-supermodel/requirements.html#_model_mappings)

Canonical Examples of system-independent data that do and do not conform to this Shape are presented within this Scenario Demonstrator and also indicated in A18's details in the table above. The example files are:

* [A18-01-v.ttl](https://github.com/nicholascar/qsi-sd-canonical/blob/master/canonical-examples/A18-01-v.ttl) - valid example 01
* [A18-02-i.ttl](https://github.com/nicholascar/qsi-sd-canonical/blob/master/canonical-examples/A18-01-v.ttl) - invalid example 02

Just for this example, the data for `A18-01-v.ttl` can be display be executing the following:

In [ ]:
from rdflib import Graph
g = Graph().parse("canonical-examples/A18-01-v.ttl")
print(g.serialize())

_Note the components "20", "Oxford", "Place" & example LGA `<http://example.com/lga/1234>`_

#### 2.1.2 Demonstration Steps

##### 1. Validate 01 - expect True

In [41]:
from pyshacl import validate

v1 = validate("canonical-examples/A18-01-v.ttl", shacl_graph="validators/addr.shacl.ttl", inference="rdfs")

if v1[0]:
    print("A18-01-v is valid, as expected")
else:
    print("ERROR: A18-01-v is unexpectedly invalid")

A18-01-v is valid, as expected


##### 2. Validate 02 - expect False

In [45]:
from pyshacl import validate

v2 = validate("canonical-examples/A18-02-i.ttl", shacl_graph="validators/addr.shacl.ttl")

if not v2[0]:
    print("A18-02-i is invalid, as expected")
    print("The error message from the validator is:")
    print()
    print(v2[2])
else:
    print("ERROR: A18-02-i is unexpectedly valid")

A18-02-i is invalid, as expected
The error message from the validator is:

Validation Report
Conforms: False
Results (2):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: <http://w3id.org/profile/anz-address/validator/sh-02>
	Focus Node: [ addr:hasComponentType isov1:locality ; addr:hasValue Literal("Shorncliffe") ]
	Value Node: Literal("Shorncliffe")
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("The hasValue property of an AddressComponent with addressComponentType isov1:locality must indicate an IRI (of a Concept from a vocabulary), not text") ; sh:prefixes <http://w3id.org/profile/anz-address/>, <https://w3id.org/profile/qsi-supermodel/> ; sh:select Literal("
			SELECT $this ?value
			WHERE {
			    {
                    $this addr:hasComponentType isov1:locality .
                    $this addr:hasValue ?value .
                    FILTER (!isIRI(?value))
			  

An inspection of the validation error above allows you to work out which `AddressComponent` object is the invalid one (there are 2!).

##### 3. Query 01

Here we query the valid example, 01, to assemble a printable address from the Address Components. This isn't necessary for Requirement A18 but shows SPARQL queries in action agains Canonical Example data.

In [40]:
from rdflib import Graph
g = Graph().parse("canonical-examples/A18-01-v.ttl")
q = """
    PREFIX addr: <http://w3id.org/profile/anz-address/>

    SELECT ?ct ?v
    WHERE {
        ?a addr:hasAddressComponent ?ac .
        ?ac addr:hasComponentType ?ct .
        ?ac addr:hasValue ?hv .

        OPTIONAL {
            ?ac addr:hasValueText ?htv .
        }

        BIND (COALESCE(?htv, ?hv) AS ?v)
    }
"""
COMP = "http://def.isotc211.org/iso19160/-1/2015/Address/code/AddressComponentType/"
components = {}
for r in g.query(q):
    components[str(r["ct"].replace(COMP, ""))] = str(r["v"])

print(f"""
{components["streetNumber"]} {components["thoroughfareName"]} {components["thoroughfareType"]}
{components["locality"]}, {components["state"]}
{components["postcode"]}
""")


20 Oxford Place
Shorncliffe, Queensland
4017



### 2.1 Req. xxx

**ID** | **Statement** | **Relevant Model Elements** | **Canonical Examples** | **Scenario Demonstrator**
--- | --- | --- | --- | ---
? | ? | ? | ? | ?
